# Oscilloskope utility – using Ethernet

In [4]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run "Record" on the oscilloscope
# and wait for 508 frames to be acquired.
# Step 2:
# Run this cell to initialize grabbing.


# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
        
# Default oscilloscope record timeout [s]
loop_sleep_time = 60
        
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::10.9.9.100::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::10.9.9.100::INSTR
RIGOL TECHNOLOGIES,DS2072A,DS2D204202923,00.03.05.SP4


## Read repeatedly records from oscilloscope
This should be run after the initialization step. Timeout at the end should be enlarged if not all 508 frames are transferred.

In [7]:
import os
import glob
import time
import h5py
import numpy as np
import sys

filename = 1

'''
if filename == 1:
    for f in glob.iglob("./data/*.h5"):
        print('Deleting', f)
        os.remove(f)
else:
    print('Not removing old files, as filename {} is not 1.'.format(filename))
'''

osc.write(':STOP')
time.sleep(0.5)

start_wfd = 0.005
wfd = start_wfd

while True:
    osc.write(':FUNC:WREC:OPER REC')
    run_start_time = time.time()
    print('  Capturing...')
    time.sleep(0.5)

    while True:
        osc.write(':FUNC:WREC:OPER?')
        reply = osc.read()
        if reply == 'STOP':
            run_time = round(time.time() - run_start_time, 2)
            print('  Subrun finished, capturing for {:.2f} seconds.'.format(run_time))
            break
        time.sleep(0.01)

    channel = 'CHAN1'
    if osc.ask(':' + channel + ':DISP?') == '0':
        print(channel + ' is not enabled')
    else:
        print('Reading out ' + channel)
        osc.write(':WAV:SOUR ' + channel)
        osc.write(':WAV:MODE NORM')
        osc.write(':WAV:FORM BYTE')
        osc.write(':WAV:POIN 1400')

        osc.write(':WAV:XINC?')
        xinc = float(osc.read(100))
        print('XINC:', xinc)
        osc.write(':WAV:YINC?')
        yinc = float(osc.read(100))
        print('YINC:', yinc)
        osc.write(':TRIGger:EDGe:LEVel?')
        trig = float(osc.read(100))
        print('TRIG:', trig)
        osc.write(':WAVeform:YORigin?')
        yorig = float(osc.read(100))
        print('YORIGIN:', yorig)
        osc.write(':WAVeform:XORigin?')
        xorig = float(osc.read(100))
        print('XORIGIN:', xorig)
        osc.write(':FUNC:WREP:FEND?')
        frames = int(osc.read(100))
        print('FRAMES:', frames, 'SUBRUN', filename)

        lastwave = bytearray()

        with h5py.File(f'./data/osc0_{filename:02d}_{int(round(run_start_time, 0))}-{channel}.h5', 'w') as hf:
            hf.create_dataset('FRAMES', data=frames)
            hf.create_dataset('XINC', data=xinc)
            hf.create_dataset('YINC', data=yinc)
            hf.create_dataset('TRIG', data=trig)
            hf.create_dataset('YORIGIN', data=yorig)
            hf.create_dataset('XORIGIN', data=xorig)
            hf.create_dataset('CAPTURING', data=run_time)
            osc.write(':FUNC:WREP:FCUR 1')
            time.sleep(0.5)

            for n in range(1, frames + 1):
                osc.write(f':FUNC:WREP:FCUR {n}')
                while True:
                    time.sleep(0.05)
                    fcur = osc.ask(':FUNC:WREP:FCUR?')
                    if str(n) == fcur:
                        sys.stdout.write(str(n))
                        break
                    else:
                        print(f"Needwait: {n} vs {fcur}")

                reread_count = 0
                while True:
                    time.sleep(wfd)
                    osc.write(':WAV:DATA?')
                    time.sleep(wfd)
                    wave1 = bytearray(osc.read_raw(500))
                    wave2 = bytearray(osc.read_raw(500))
                    wave3 = bytearray(osc.read_raw(500))
                    wave = np.concatenate((wave1[11:], wave2, wave3[:-1]))

                    if np.array_equal(wave, lastwave):
                        wfd += 0.005
                        print(f' Same waveform, wait {wfd:.3f} and reread')
                        reread_count += 1
                        if reread_count > 5:
                            print('------------ Wrong trigger level?')
                    else:
                        hf.create_dataset(str(n), data=wave)
                        lastwave = wave
                        sys.stdout.write('.')
                        wfd = start_wfd
                        break

        print('OK')
    filename += 1


  Capturing...
  Subrun finished, capturing for 3.18 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 1
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.82 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 2
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 2.61 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 3
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 2.93 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 4
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.79 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 5
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, captur

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 7.01 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 44
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.47 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 45
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 2.66 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 46
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.20 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 47
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.74 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 48
1.2.3.4.5.6.7.OK
  Capturing...
  S

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.73 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 87
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.72 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 88
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.46 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 89
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.38 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 90
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.19 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 91
1.2.3.4.5.6.7.OK
  Capturing...
  S

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.39 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 130
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.30 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 131
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.74 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 132
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 6.40 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 133
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 7.34 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 134
1.2.3.4.5.6.7.OK
  Capturing..

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.09 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 173
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 9.81 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 174
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.32 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 175
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 6.89 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 176
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.20 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 177
1.2.3.4.5.6.7.OK
  Capturing..

...

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.02 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 774
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.76 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 775
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.19 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 776
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.45 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 777
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.81 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 778
1.2.3.4.5.6.7.OK
  Capturing..

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.63 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 817
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.83 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 818
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 2.00 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 819
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.01 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 820
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.10 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 821
1.2.3.4.5.6.7.OK
  Capturing..

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 5.81 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 860
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.36 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 861
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 8.27 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 862
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 6.61 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 863
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.54 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 864
1.2.3.4.5.6.7.OK
  Capturing..

1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 3.94 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 903
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.55 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 904
1.2.3.4.5.6.7.OK
  Capturing...
  Subrun finished, capturing for 4.95 seconds.
Reading out CHAN1
XINC: 2e-07
YINC: 0.0004
TRIG: 0.0058
YORIGIN: -53.0
XORIGIN: -3.959999e-05
FRAMES: 7 SUBRUN 905
1.2.3.4.5.6.7.OK
  Capturing...


KeyboardInterrupt: 